# 4.1.6-使用ArcGIS-Pro和Arcy制作建筑年代预测结果在150米精度范围的准确度

我们要复现论文中的图8：

图片上表现的是预测的精准程度在150m的网格上的空间分布，图中可以看出：市中心的错误率高于郊区。这可能是由于建筑年龄的高度多样性、市中心旧建筑的频繁改造以及上述对改造建筑的严格规定所致。阿姆斯特丹郊区没有明显的空间格局，这表明分类结果的空间相关性很小。为了证明空间相关性小，作者还计算了莫兰指数，城市郊区结果的 Moran's I 为 0.27。 Moran's I 的范围为 -1 到 1，其中 -1 表示完美分散`dispersion` ，0 表示完美随机性  `randomness`，1 表示完美聚类`clustering`。我们预测的阿姆斯特丹郊区的莫兰指数为0.27，建筑年代表现出较弱的空间自相关性。

In [1]:
# 导入arcpy并设置工作空间
import arcpy
import geopandas as gpd
import pandas as pd

# 设置工作空间
gdb = r"C:\Users\hncdj\Documents\Python_\Python辅助城市研究\Transportation-data-analysis\ArcGIS_Pro-Project\Transportation-data-analysis\Transportation-data-analysis.gdb" # 
arcpy.env.workspace = gdb
arcpy.env.overwriteOutput = True

# 1读取出粗车数据 构建起点坐标的geodataframe

In [2]:
data = pd.read_csv('../data/trips-clean.csv', 
                   header=0, 
                #    date_format='%Y-%m-%d %H:%M:%S', 
                #    parse_dates=['StartTime', 'EndTime']
                   )
data.head()


,VehicleNum,StartTime,EndTime,StartLng,StartLat,EndLng,EndLat,TripTime,TripDistance
0,22223.0,2013-10-22 00:03:39,2013-10-22 00:11:04,114.167732,22.562550,114.225487,22.552933,7.416667,6.035011
1,22223.0,2013-10-22 00:11:49,2013-10-22 00:15:35,114.227264,22.554234,114.229202,22.560150,3.766667,0.684779
2,22223.0,2013-10-22 00:17:29,2013-10-22 00:29:21,114.231598,22.562349,114.255898,22.591000,11.866667,4.038736
3,22223.0,2013-10-22 00:37:01,2013-10-22 00:54:58,114.239700,22.563450,114.119637,22.565916,17.950000,12.351299
4,22223.0,2013-10-22 01:01:29,2013-10-22 01:08:33,114.136467,22.575583,114.166786,22.608232,7.066667,4.774048


In [3]:
# 构建gdf
geo_data = gpd.GeoDataFrame(data, geometry=gpd.points_from_xy(data['StartLng'], data['StartLat']))
geo_crs = '4326'
transform_crs = '32649'
geo_data.crs = geo_crs
geo_data.to_crs(transform_crs, inplace=True)
# geo_data.to_file(filename=gdb, layer='trips')

## 2.利用ArcPy的创建渔网工具创建150米的空间网格

### 2.1创建渔网函数解析


1. 官方帮助文档：[创建渔网 (数据管理)](https://pro.arcgis.com/zh-cn/pro-app/3.0/tool-reference/data-management/create-fishnet.htm)
2. 在[《利用ArcGIS_Python制作考虑路况的交通等时圈》](https://cdn.renhai-lab.tech/archives/4.2.14-%E5%AE%9E%E6%93%8D3-%E5%88%A9%E7%94%A8ArcGIS_Python%E5%88%B6%E4%BD%9C%E8%80%83%E8%99%91%E8%B7%AF%E5%86%B5%E7%9A%84%E4%BA%A4%E9%80%9A%E7%AD%89%E6%97%B6%E5%9C%88#3.%E5%88%9B%E5%BB%BA%E6%B8%94%E7%BD%91)文也讲过：点击文章查看, 本文中的代码是在此基础上修改的。关键信息如下：
- **a.创建渔网函数的参数**：
    - out_feature_class：包含由矩形像元组成的渔网的输出要素类。
    - **origin_coord**：矩形框的左下端点为原点
    - **y_axis_coord**：此点与原点的连线用于判断旋转的角度 我们不用旋转所以定义为原点正上方的点
    - cell_width：网格的宽度
    - cell_height：网格的高度
    - number_rows：填'0',代表留空，由cell_width和cell_height决定
    - number_columns：填'0'，由cell_width和cell_height决定
    - corner_coord：填'0'
    - **labels**：'LABELS'
    - **template**：以空格分隔的坐标字符串 - 将使用指定渔网的范围。坐标以 x-min，y-min，x-max，y-max 的顺序表示。
    - geometry_type：生成面
- **b.创建渔网返回的结果**：
    - out_feature_class：包含由矩形像元组成的渔网的输出要素类。
    - out_label：创建一个新的点要素类，其中每个渔网像元中心都具有标注点。如果选中了创建标注点参数（Python 中的 labels = 'LABELS'），则会创建一个新的点要素类，其中每个渔网像元中心都具有标注点。此要素类的名称以 _label 为后缀并与输出要素类相同，且创建于同一位置。

### 2.2 定义origin_coord和y_axis_coord

#### 从shenzhen的shp文件读取出四周边界

In [4]:
from shapely.geometry import box
import geopandas as gpd

file = r'C:\Users\hncdj\Documents\Python_\Python辅助城市研究\Transportation-data-analysis\【专题一】出租车GPS数据分析\第6章-出租车 GPS 数据-时空大数据处理基础\data\sz\sz.shp'
shenzhen = gpd.read_file(file, encoding='utf-8', engine='fiona')
shenzhen.set_crs(geo_crs, inplace=True)
shenzhen.to_crs(transform_crs, inplace=True)
# total_bounds返回一个元组，包含最小的x，最小的y，最大的x，最大的y

x_min, y_min, x_max, y_max = shenzhen.total_bounds
# 创建一个边界框
bbox_geometry = box(x_min, y_min, x_max, y_max)
# 创建一个GeoDataFrame，将边界框作为几何图形
gdf_bbox = gpd.GeoDataFrame({'geometry': bbox_geometry}, index=[0]).set_crs(32649)
gdf_bbox

,geometry
0,"POLYGON ((872889.114 2479737.909, 872889.114 2..."


In [5]:
# 定义origin_coord和y_axis_coord 空格分割
origin_coord = f"{x_min}  {y_min}"
y_axis_coord = f"{x_min}  {y_max}"
origin_coord, y_axis_coord

('782100.025087936  2479737.9085472594', '782100.025087936  2531444.336644171')

### arcpy创建渔网

In [9]:
scales = 250 # 网格的单元尺度 单位为米

In [10]:
extent = f"{x_min} {y_min} {x_max} {y_max}"
extent

'782100.025087936 2479737.9085472594 872889.114095384 2531444.336644171'

In [12]:
# 设置空间参考对象
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference(int(transform_crs))

In [16]:
out_fcs = 'fishnet250m' # 有bug 不能数字开头

In [17]:
# 设置工作空间
arcpy.management.CreateFishnet(out_feature_class = out_fcs, # 包含由矩形像元组成的渔网的输出要素类。
                               origin_coord = origin_coord, # 矩形框的左下端点为原点
                               y_axis_coord = y_axis_coord, # 此点与原点的连线用于判断旋转的角度 我们不用旋转所以定义为原点正上方的点
                               cell_width = scales, 
                               cell_height = scales,
                               number_rows = "0", # 留空，由cell_width和cell_height决定
                               number_columns = "0", # 留空，由cell_width和cell_height决定
                               corner_coord = None, # 对角坐标不填写
                               labels = "LABELS", 
                               template = extent, # 以空格分隔的坐标字符串 - 将使用指定渔网的范围。坐标以 x-min，y-min，x-max，y-max 的顺序表示。
                               geometry_type = "POLYGON" # 生成面
                               )

<Result 'C:\\Users\\hncdj\\Documents\\Python_\\Python辅助城市研究\\Transportation-data-analysis\\ArcGIS_Pro-Project\\Transportation-data-analysis\\Transportation-data-analysis.gdb\\fishnet250m'>

In [18]:
# 定义新产生的点要素的名称
out_label = out_fcs + "_label"

## 3.空间连接

geopandas和arcpy都有空间连接的功能，但是geopandas的空间连接功能更强大，而且方便进行数据统计，所以我们使用geopandas的空间连接功能。

在选择处理工具的时候，我不喜欢用arcpy的游标进行数据处理，因为arcpy的游标处理数据的速度太慢了，而且代码也不够简洁。所以我更喜欢用geopandas基于dataframe的处理，更强大也更简洁。
但是arcpy中提供一些实用的工具，比如同时创建渔网和渔网标注点，这个功能在geopandas中没有，所以我们用arcpy创建渔网，然后用geopandas进行空间连接。另外，制图的时候，我们也会用到ArcGIS Pro，可以实时看到制图的效果，并且渲染效果更好。

In [19]:
# 读取渔网
gdf_fishnet = gpd.read_file(filename=gdb, layer=out_fcs)

# 读取渔网标注点
gdf_fishnet_label = gpd.read_file(filename=gdb, layer=out_label)

In [20]:
len(gdf_fishnet), len(gdf_fishnet_label)

(75348, 75348)

In [21]:
# 给渔网和标注点添加id列
gdf_fishnet['id'] = gdf_fishnet.index
gdf_fishnet_label['id'] = gdf_fishnet_label.index

In [22]:
# 查看geopandas的版本
gpd.__version__

'0.14.0'

### 空间连接的方法：一种是使用geopandas.GeoDataFrame.sjoin_nearest 空间连接，一种是使用geopandas.GeoDataFrame.sjoin 空间连接

- [官方帮助文档](https://geopandas.org/en/latest/docs/user_guide/mergingdata.html#spatial-joins)


关键参数：
1. predicate 值，这些值在 [Shapely文档](http://shapely.readthedocs.io/en/latest/manual.html#binary-predicates) 中定义：

    intersects 相交
    
    contains 包含
    
    within 之内
    
    touches 触摸
    
    crosses 十字架
    
    overlaps 重叠
how：
2. how 参数指定将发生的连接类型以及在结果 GeoDataFrame 中保留哪些几何图形。它接受以下选项：
    left ：使用您提供给 GeoDataFrame.sjoin() 的第一个（或 left_df） GeoDataFrame 中的索引；仅保留 left_df 几何列
    
    right ：使用第二个（或 right_df）的索引；仅保留 right_df 几何列
    
    inner ：使用两个 GeoDataFrame 索引值的交集；仅保留 left_df 几何列

In [23]:
# 我们已经有方格网了，直接使用geopandas.GeoDataFrame.sjoin 空间连接, 保留左边的数据，即渔网的数据，然后根据右侧的数据进行计算
gdf_fishnet = gpd.sjoin(gdf_fishnet, geo_data, how='left', predicate='intersects') # op参数在将来的版本中将被弃用，并建议您使用predicate参数代替。
len(gdf_fishnet)

562286

In [24]:
gdf_fishnet.head()

,Shape_Length,Shape_Area,geometry,id
0,600.0,22500.0,"MULTIPOLYGON (((782100.025 2479737.909, 782100...",0
1,600.0,22500.0,"MULTIPOLYGON (((782250.025 2479737.909, 782250...",1
2,600.0,22500.0,"MULTIPOLYGON (((782400.025 2479737.909, 782400...",2
3,600.0,22500.0,"MULTIPOLYGON (((782550.025 2479737.909, 782550...",3
4,600.0,22500.0,"MULTIPOLYGON (((782700.025 2479737.909, 782700...",4


In [27]:
# 扔掉空网格数据
gdf_fishnet = gdf_fishnet.dropna(subset=['index_right'])
gdf_fishnet.head()

,Shape_Length,Shape_Area,geometry,id,index_right,VehicleNum,StartTime,EndTime,StartLng,StartLat,EndLng,EndLat,TripTime,TripDistance
1514,1000.0,62500.0,"MULTIPOLYGON (((796600.025 2480737.909, 796600...",1514,311584.0,31830.0,2013-10-22 19:05:17,2013-10-22 19:11:14,113.883034,22.409233,114.057350,22.537216,5.950000,22.862581
2300,1000.0,62500.0,"MULTIPOLYGON (((811100.025 2481237.909, 811100...",2300,310178.0,31784.0,2013-10-22 23:14:30,2013-10-22 23:34:24,114.022903,22.409550,114.147514,22.375467,19.900000,13.375424
3076,1000.0,62500.0,"MULTIPOLYGON (((823100.025 2481737.909, 823100...",3076,310180.0,31784.0,2013-10-22 23:39:12,2013-10-22 23:46:09,114.138580,22.412901,114.107620,22.481182,6.950000,8.205350
4151,1000.0,62500.0,"MULTIPOLYGON (((818850.025 2482487.909, 818850...",4151,311511.0,31827.0,2013-10-22 18:51:32,2013-10-22 19:02:09,114.098000,22.419634,114.020531,22.531816,10.616667,14.760962
7373,1000.0,62500.0,"MULTIPOLYGON (((805350.025 2484737.909, 805350...",7373,382468.0,33801.0,2013-10-22 17:47:56,2013-10-22 18:45:03,113.967171,22.443167,113.949402,22.580584,57.116667,15.326608


## 根据需要计算的字段进行分组

In [ ]:
# 计算每个网格内的总出行次数
gdf_fishnet_trips = gdf_fishnet.groupby('id').size().reset_index(name='trips')
gdf_fishnet_trips.head()

In [28]:
# 正则化到0-1
gdf_fishnet_trips['trips'] = gdf_fishnet_trips['trips'] / gdf_fishnet_trips['trips'].max()
gdf_fishnet_trips.head()


,id,trips
0,1514,0.000234
1,2300,0.000234
2,3076,0.000234
3,4151,0.000234
4,7373,0.000234


In [29]:
# 与原始网格数据进行连接
trips_250m = gdf_fishnet.merge(gdf_fishnet_trips, on='id', how='left')

In [31]:
trips_250m.head()

,Shape_Length,Shape_Area,geometry,id,index_right,VehicleNum,StartTime,EndTime,StartLng,StartLat,EndLng,EndLat,TripTime,TripDistance,trips
0,1000.0,62500.0,"MULTIPOLYGON (((796600.025 2480737.909, 796600...",1514,311584.0,31830.0,2013-10-22 19:05:17,2013-10-22 19:11:14,113.883034,22.409233,114.057350,22.537216,5.950000,22.862581,0.000234
1,1000.0,62500.0,"MULTIPOLYGON (((811100.025 2481237.909, 811100...",2300,310178.0,31784.0,2013-10-22 23:14:30,2013-10-22 23:34:24,114.022903,22.409550,114.147514,22.375467,19.900000,13.375424,0.000234
2,1000.0,62500.0,"MULTIPOLYGON (((823100.025 2481737.909, 823100...",3076,310180.0,31784.0,2013-10-22 23:39:12,2013-10-22 23:46:09,114.138580,22.412901,114.107620,22.481182,6.950000,8.205350,0.000234
3,1000.0,62500.0,"MULTIPOLYGON (((818850.025 2482487.909, 818850...",4151,311511.0,31827.0,2013-10-22 18:51:32,2013-10-22 19:02:09,114.098000,22.419634,114.020531,22.531816,10.616667,14.760962,0.000234
4,1000.0,62500.0,"MULTIPOLYGON (((805350.025 2484737.909, 805350...",7373,382468.0,33801.0,2013-10-22 17:47:56,2013-10-22 18:45:03,113.967171,22.443167,113.949402,22.580584,57.116667,15.326608,0.000234


In [32]:
cols_to_keep = ['id', 'TripTime', 'TripDistance', 'trips']
trips_250m[cols_to_keep]

,id,TripTime,TripDistance,trips
0,1514,5.950000,22.862581,0.000234
1,2300,19.900000,13.375424,0.000234
2,3076,6.950000,8.205350,0.000234
3,4151,10.616667,14.760962,0.000234
4,7373,57.116667,15.326608,0.000234
...,...,...,...,...
495224,74103,36.233333,13.142940,0.000234
495225,74127,22.033333,13.872942,0.000234
495226,74300,23.500000,24.437779,0.000234
495227,74317,8.116667,13.596781,0.000234


In [33]:
# 我们需要的是点的准确度，所以将渔网的准确度赋值给渔网标注点 并且需要扔掉空值 我们使用merge
gdf_fishnet_point = gdf_fishnet_label.merge(trips_250m[cols_to_keep], left_on='id',right_on='id' , how='right')
gdf_fishnet_point

,geometry,id,TripTime,TripDistance,trips
0,POINT (796725.025 2480862.909),1514,5.950000,22.862581,0.000234
1,POINT (811225.025 2481362.909),2300,19.900000,13.375424,0.000234
2,POINT (823225.025 2481862.909),3076,6.950000,8.205350,0.000234
3,POINT (818975.025 2482612.909),4151,10.616667,14.760962,0.000234
4,POINT (805475.025 2484862.909),7373,57.116667,15.326608,0.000234
...,...,...,...,...,...
495224,POINT (834975.025 2530612.909),74103,36.233333,13.142940,0.000234
495225,POINT (840975.025 2530612.909),74127,22.033333,13.872942,0.000234
495226,POINT (793225.025 2530862.909),74300,23.500000,24.437779,0.000234
495227,POINT (797475.025 2530862.909),74317,8.116667,13.596781,0.000234


In [34]:
### 保存点数据
accuracy_points = gdf_fishnet_point.drop(columns=['id'])
gpd.GeoDataFrame(accuracy_points, geometry='geometry').set_crs(transform_crs).to_file(filename=gdb, layer='accuracy_250m_points')

<ipython-input-34-287389baec70>:3: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gpd.GeoDataFrame(accuracy_points, geometry='geometry').set_crs(transform_crs).to_file(filename=gdb, layer='accuracy_250m_points')


## ArcGIS Pro制图
详见ArcGIS Pro工程

![](../../5-ArcgisPro工程/阿姆斯特丹全市范围建筑年代预测准确度空间分布图.jpg)